In [1]:
rpc_info = {}
def xmlrpc(in_=(), out_=(type(None), )):
    def _xmlrpc(func):
        # register the signature
        func_name = func.__name__
        rpc_info[func_name] = (in_, out_)
        def _check_types(elements, types):
            """Subfunction that checks the types"""
            if len(elements) != len(types):
                raise TypeError("argument count is wrong")
            for index, (arg, type_) in enumerate(zip(elements, types)):
                if isinstance(arg, type_):
                    continue
                raise TypeError("arg #%d should be %s" % (index, type_))
        # wrapped function
        def __xmlrpc(*args):  # no keywords allowed
            checkable_args = args[1:]  # remove self
            _check_types(checkable_args, in_)
            # running function
            res = func(*args)
            # checking output
            if not type(res) in (tuple, list):
                checkable_res = (res, )
            else:
                checkable_res = res
            _check_types(checkable_res, out_)
            
            # both input and output types are valid
            return res
        return __xmlrpc
    return _xmlrpc

In [2]:
class RPCView:
    @xmlrpc((int, int))  # (int, int) -> None
    def meth1(self, int1, int2):
        print("received %d and %d" % (int1, int2))
    @xmlrpc((str, ), (int, ))
    def meth2(self, phrase):
        print("received %s" % phrase)
        return 12

In [3]:
rpc_info

{'meth1': ((int, int), (NoneType,)), 'meth2': ((str,), (int,))}

In [4]:
my = RPCView()

In [5]:
my.meth1(1, 2)

received 1 and 2


In [6]:
my.meth2(2)  # will raise error

TypeError: arg #0 should be <class 'str'>